In [53]:
PATH = "/Users/joregan/Playing/hu-tts/hungarian-single-speaker-tts"

In [ ]:
!pip install datasets librosa

In [55]:
from pathlib import Path
import datasets

In [56]:
path = Path(PATH)
outdir = path / "data"
transcript = path / "transcript.txt"

In [57]:
def clean_text(text, norm = False):
    if text.endswith(" -"):
        text = text[:-2]
        text = text.replace(".S ", ". S ")
    if norm:
        text = text.replace(". ", " ")
        if len(text) > 1 and text[-1] in [".", "!", "?", ":"]:
            text = text[:-1]
        text = text.replace(" -, ", " ")
        text = text.replace(" - ", " ")
        text = text.replace(". ", " ")
        text = text.replace(", ", " ")
        text = text.replace("? ", " ")
        text = text.replace(": ", " ")
        text = text.replace("! ", " ")
        text = text.lower()
    return text

In [58]:
data = []
with open(str(transcript)) as ts:
    for line in ts.readlines():
        data.append(line.strip().split("|"))

In [59]:
def data_gen():
    for i in data:
        filepath = i[0]
        fileid = filepath.split("/")[1].replace(".wav", "")
        text = i[1]
        lightly_cleaned = clean_text(text)
        fully_cleaned = clean_text(text, True)
        yield {
            "id": fileid,
            "audio": str(path / filepath),
            "original_text": lightly_cleaned,
            "text": fully_cleaned,
            "duration": float(i[3])
        }

In [73]:
from datasets import Dataset, Audio
dataset = Dataset.from_generator(data_gen).cast_column("audio", Audio(sampling_rate=22050))

Using custom data configuration default-2ea64471c62bb335
/Users/joregan/opt/anaconda3/envs/hf/lib/python3.9/site-packages/datasets/builder.py:712: FutureWarning: 'use_auth_token' was deprecated in version 2.7.1 and will be removed in 3.0.0. Pass `use_auth_token` to the initializer/`load_dataset_builder` instead.
  warnings.warn(
Found cached dataset generator (/Users/joregan/.cache/huggingface/datasets/generator/default-2ea64471c62bb335/0.0.0)


In [75]:
dataset.push_to_hub("KTH/hungarian-single-speaker-tts", max_shard_size="500MB")

Deleting unused files from dataset repository: 100%|██████████| 7/7 [02:18<00:00, 19.81s/it]


StopIteration: 